# Create MongoDB database

In [1]:
from pymongo import MongoClient
import os
import json
import re
import pandas as pd

In [2]:
# Create a client connection to the MongoDb instance running on the local machine
client = MongoClient('localhost:27017')

In [3]:
# define a database object to insert data into the MongoDb database
db = client.Awards

In [4]:
one_file_dir = os.path.join(os.pardir,'data', 'interim', '2018.json')
with open(one_file_dir) as f:
    year = json.load(f)

In [6]:
len(year), year[0]

(19,
 {'title': 'Validation of a Performance Based Measure of Functioning in MCI and Early AD ',
  'Agency': 'Department of Health and Human Services',
  'Branch': 'National Institutes of Health',
  'Contract': '4R44AG058345-02',
  'Agency Tracking Number': 'R44AG058345',
  'Socially and Economically Disadvantaged': 'N',
  'Amount': '$946,528.00',
  'Phase': 'Phase I',
  'Program': 'SBIR',
  'Awards Year': '2018',
  'Solicitation Year': '2017',
  'Solicitation Topic Code': 'NIA',
  'Solicitation Number': 'PAS17-064',
  'HUBZone Owned': 'N',
  'Woman Owned': 'Y',
  'company name': 'NEUROCOG TRIALS',
  'company address': '1709 LEGION RD STE 221, Chapel Hill, NC, 27517-2374',
  'DUNS': '175615165',
  'Principal Investigator': {'Name': 'CAREN GADIGIAN',
   'Phone': '(919) 401-4642',
   'Email': 'alexandra.atkins@neurocogtrials.com'},
  'Business Contact': {'Name': 'CAREN GADIGIAN',
   'Phone': '(919) 401-4642',
   'Email': 'caren.gadigian@neurocogtrials.com'},
  'Research Institution': 'N/

### Convert dollar amount to integer

In [65]:
# convert amount to integer
for y in year:
    amount_str = y.get('Amount', 'N/A')
    amount_digit = re.sub('[^\d.]', '', amount_str)
    # check if there are any numbers in amount
    if not not amount_digit:
        y['Amount'] = float(amount_digit)

### Retrieve all awards (documents)

In [7]:
# cursor is a generator (loop through it)
cursor = db.Awards.find({})

In [8]:
next(cursor)

{'_id': ObjectId('5b21be35cbf6d31f2ed0494b'),
 'title': 'Validation of a Performance Based Measure of Functioning in MCI and Early AD ',
 'Agency': 'Department of Health and Human Services',
 'Branch': 'National Institutes of Health',
 'Contract': '4R44AG058345-02',
 'Agency Tracking Number': 'R44AG058345',
 'Socially and Economically Disadvantaged': 'N',
 'Amount': '$946,528.00',
 'Phase': 'Phase I',
 'Program': 'SBIR',
 'Awards Year': '2018',
 'Solicitation Year': '2017',
 'Solicitation Topic Code': 'NIA',
 'Solicitation Number': 'PAS17-064',
 'HUBZone Owned': 'N',
 'Woman Owned': 'Y',
 'company name': 'NEUROCOG TRIALS',
 'company address': '1709 LEGION RD STE 221, Chapel Hill, NC, 27517-2374',
 'DUNS': '175615165',
 'Principal Investigator': {'Name': 'CAREN GADIGIAN',
  'Phone': '(919) 401-4642',
  'Email': 'alexandra.atkins@neurocogtrials.com'},
 'Business Contact': {'Name': 'CAREN GADIGIAN',
  'Phone': '(919) 401-4642',
  'Email': 'caren.gadigian@neurocogtrials.com'},
 'Abstract':

### Remove fields with the following missing values:
- { 'Research Institution': 'N/A' }

In [ ]:
# update by removing 'Research Institution' field where 'Research Institution' is N/A
# PS: 'Research Institution' asociated value after unset can be anything (1,0,"",...)
db.Awards.update_many(
   { 'Research Institution': 'N/A' },\
   {'$unset': {'Research Institution':1}}
    )

### Return only one field without value restrictions

In [14]:
cursor_tmp = db.Awards.find({'Solicitation Year' : {'$exists': True}}, \
                            {'Solicitation Year' : 1, '_id': False})      # projection (Only returns those fields)
# cursor_tmp = db.Awards.find({'Solicitation Year' : 1})

In [15]:
next(cursor_tmp)

[{'Solicitation Year': '2017'},
 {'Solicitation Year': '2015'},
 {'Solicitation Year': '2015'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2015'},
 {'Solicitation Year': '2015'},
 {'Solicitation Year': '2015'},
 {'Solicitation Year': '2014'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2015'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2016'},
 {'Solicitation Year': '2017'},
 {'Solicitation Year': '2017'}]

### Distinct value of one field over all documents (collection)

In [34]:
# Find distinct values for a given field
list(db.Awards.distinct('Solicitation Year'))

['2017', '2015', '2016', '2014']